In [ ]:
!pip install catboost

     |████████████████████████████████| 65.9MB 58kB/s 


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import gc

In [ ]:
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_ = pd.read_csv('Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [ ]:
from sklearn.model_selection import KFold

def get_train_test_names(train_, test_, submission_):
  kf = KFold(n_splits=5, shuffle=False)
  for r, (train_index, test_index) in enumerate(kf.split(train_)):
    test = train_.iloc[test_index]

    X_test = []
    X_test_columns = test.columns
    for v in test.values:
      info = v[:8]
      binary = v[8:]
      index = [k for k, i in enumerate(binary) if i == 1]
      for i in index:
        for k in range(len(binary)):
          if k == i:
            binary_transformed = list(copy.copy(binary))
            binary_transformed[i] = 0
            X_test.append(list(info) + binary_transformed)

    X_test = pd.DataFrame(X_test)
    X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

    X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

    yield train_.iloc[train_index], X_test, submission_, '2_fold' + str(r) + '.csv'
  yield train_, test_, submission_, '2_main.csv'

In [ ]:
for train, test, submission, name in get_train_test_names(train_, test_, submission_):

  true_values = []
  for v in test.values:
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in test.columns[8:][index]:
      true_values.append(v[0] + ' X ' + k)

  #Store Products ID in Multidimensional array
  products =train[['P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
  products_t =test[['P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

  #Axis Transformation
  train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
  test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

  #A merger
  train['combiner']='x'

  #Set
  data=pd.concat([train,test],sort=False).reset_index(drop=True)

  train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
  train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)

  #Concatenate new column towards submission requirement
  train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

  #Re-arrange column (not necessary)
  train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
  test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']]


  train['date1'] = train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
  train['date2'] = train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
  train['date3'] = train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
  train.drop('join_date', axis=1, inplace=True)

  test['date1'] = test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
  test['date2'] = test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
  test['date3'] = test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
  test.drop('join_date', axis=1, inplace=True)

  train['date_diff'] = train['date3'] - train['birth_year']
  test['date_diff'] = test['date3'] - test['birth_year']

  model = CatBoostClassifier()
  model.fit(train.drop(columns=['ID', 'Label', 'ID X PCODE']), train['Label'], cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code', 'PCODE'])

  proba = model.predict_proba(test.drop(columns=['ID','ID X PCODE'], axis=1))[:, 1]

  df_answer = test[['ID X PCODE']]
  df_answer['Label'] = proba

  df_answer.columns = ['ID X PCODE', 'Label']
  for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
      df_answer['Label'].iloc[i] = 1.0

  df_answer.reset_index(drop=True, inplace=True)
  df_answer.to_csv(name, index=False)

Learning rate set to 0.145018
0:	learn: 0.4840400	total: 712ms	remaining: 11m 51s
1:	learn: 0.3579291	total: 1.28s	remaining: 10m 39s
2:	learn: 0.2867817	total: 1.77s	remaining: 9m 49s
3:	learn: 0.2369447	total: 2.3s	remaining: 9m 32s
4:	learn: 0.1990301	total: 2.83s	remaining: 9m 23s
5:	learn: 0.1804115	total: 3.31s	remaining: 9m 8s
6:	learn: 0.1651434	total: 3.81s	remaining: 9m
7:	learn: 0.1535928	total: 4.27s	remaining: 8m 49s
8:	learn: 0.1469659	total: 4.79s	remaining: 8m 48s
9:	learn: 0.1411049	total: 5.3s	remaining: 8m 45s
10:	learn: 0.1378088	total: 5.83s	remaining: 8m 44s
11:	learn: 0.1343499	total: 6.26s	remaining: 8m 35s
12:	learn: 0.1327400	total: 6.73s	remaining: 8m 30s
13:	learn: 0.1306171	total: 7.17s	remaining: 8m 25s
14:	learn: 0.1295746	total: 7.59s	remaining: 8m 18s
15:	learn: 0.1280398	total: 8.01s	remaining: 8m 12s
16:	learn: 0.1269031	total: 8.4s	remaining: 8m 5s
17:	learn: 0.1259231	total: 9.01s	remaining: 8m 11s
18:	learn: 0.1252235	total: 9.49s	remaining: 8m 9s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Learning rate set to 0.145018
0:	learn: 0.4838633	total: 599ms	remaining: 9m 57s
1:	learn: 0.3583071	total: 1.14s	remaining: 9m 29s
2:	learn: 0.2876624	total: 1.67s	remaining: 9m 15s
3:	learn: 0.2337331	total: 2.17s	remaining: 9m
4:	learn: 0.2035266	total: 2.62s	remaining: 8m 40s
5:	learn: 0.1778264	total: 3.08s	remaining: 8m 29s
6:	learn: 0.1632041	total: 3.6s	remaining: 8m 30s
7:	learn: 0.1528834	total: 4.09s	remaining: 8m 27s
8:	learn: 0.1475094	total: 4.5s	remaining: 8m 15s
9:	learn: 0.1422004	total: 5.01s	remaining: 8m 16s
10:	learn: 0.1377746	total: 5.56s	remaining: 8m 19s
11:	learn: 0.1344500	total: 6.05s	remaining: 8m 18s
12:	learn: 0.1316041	total: 6.55s	remaining: 8m 17s
13:	learn: 0.1294207	total: 7.01s	remaining: 8m 13s
14:	learn: 0.1279226	total: 7.39s	remaining: 8m 5s
15:	learn: 0.1268796	total: 7.85s	remaining: 8m 3s
16:	learn: 0.1257071	total: 8.3s	remaining: 7m 59s
17:	learn: 0.1250139	total: 8.71s	remaining: 7m 55s
18:	learn: 0.1241959	total: 9.25s	remaining: 7m 57s
1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Learning rate set to 0.145021
0:	learn: 0.4885503	total: 656ms	remaining: 10m 54s
1:	learn: 0.3577186	total: 1.19s	remaining: 9m 53s
2:	learn: 0.2789647	total: 1.68s	remaining: 9m 18s
3:	learn: 0.2277757	total: 2.22s	remaining: 9m 13s
4:	learn: 0.1997461	total: 2.74s	remaining: 9m 5s
5:	learn: 0.1755870	total: 3.19s	remaining: 8m 47s
6:	learn: 0.1642151	total: 3.65s	remaining: 8m 38s
7:	learn: 0.1552956	total: 4.15s	remaining: 8m 35s
8:	learn: 0.1468382	total: 4.69s	remaining: 8m 36s
9:	learn: 0.1407663	total: 5.15s	remaining: 8m 29s
10:	learn: 0.1365790	total: 5.59s	remaining: 8m 22s
11:	learn: 0.1334304	total: 6.01s	remaining: 8m 15s
12:	learn: 0.1308040	total: 6.46s	remaining: 8m 10s
13:	learn: 0.1296053	total: 6.94s	remaining: 8m 8s
14:	learn: 0.1282691	total: 7.39s	remaining: 8m 5s
15:	learn: 0.1271481	total: 7.99s	remaining: 8m 11s
16:	learn: 0.1260049	total: 8.46s	remaining: 8m 9s
17:	learn: 0.1249962	total: 8.93s	remaining: 8m 6s
18:	learn: 0.1241460	total: 9.4s	remaining: 8m 5

OK